In [84]:
%display latex

%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile young_idempotent.py
%runfile add_degree.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

# Code

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [3]:
def vandermonde(gamma):
    n = gamma.size()
    P = DiagonalPolynomialRing(QQ, n, 1, inert=1)
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [4]:
def deriv(x, k=1):
    def f(p):
        return derivative(p, x, k)
    return f

def partial_derivatives(P):
    n = P._n
    r = P._r
    D = P._grading_set
    X = P.variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [deriv(X[i,k]) for k in range(n)]
    return op

def steenrod_operators(P, degree=1):
    r = P._r
    D = P._grading_set
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [functools.partial(P.steenrod_op, i=i, k=degree+1)]
    return op

def polarization_operators(P, side=None, row_symmetry=None):
    n = P._n
    r = P._r
    grading_set = P._grading_set
    return {grading_set([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [functools.partial(P.polarization, i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, n)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def symmetric_derivatives(P, list_deg, row_symmetry=None):
    D = P._grading_set
    return {D(-i for i in d) : [functools.partial(P.symmetric_derivative, d=d, row_symmetry=row_symmetry)] for d in list_deg}
  

def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

Projection sur les composantes isotypiques; prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [5]:
def Isotyp(S, arg):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    for nu in list_partitions:
        for key, value in basis.iteritems():
            gen = [apply_young_idempotent(p, nu) for p in value]
            basis_nu = Subspace(gen, {}).basis()
            if basis_nu != {} :
                result[(key, tuple(nu))] = basis_nu[0]
    return Subspace(result, operators={})


def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

Création de l'espace polarisé par les opérateurs donnés en paramètres.

In [6]:
def PolarizedSpace(pol_ring, S, operators):
    basis = S.basis()
    P = pol_ring
    r = P._r
    D = P._grading_set
    
    generators = {}
    for key, value in basis.iteritems():
        d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
        generators[d] = [P(b) for b in value]
    return Subspace(generators, operators, add_degrees=add_degrees_isotypic)

Calcul de l'image de S (Subspace) par les opérateurs donnés.

In [7]:
def Range(S, operators):
    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees_isotypic(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {})
    else :
        return None

Calcul du caractère à partir d'un sous-espace (S) d'éléments projetés sur les composantes isotypiques.

In [8]:
def character(S, n, r, left_basis=s, right_basis=s, row_symmetry=None):
    basis = S.basis()
    charac = 0
    if row_symmetry != "permutation":
        q = PolynomialRing(QQ,'q',r).gens()
        
    for nu in Partitions(n):
        basis_nu = {}
        charac_nu = 0
        # Get the nu_isotypic part of the basis
        for key, value in basis.iteritems():
            if Partition(key[1]) == nu:
                basis_nu[key[0]] = value
        
        # Use monomials to compute the character
        if row_symmetry == "permutation":
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(m(Partition(deg)) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                elif left_basis != m :
                    charac_nu = left_basis(charac_nu)
                
        # Or use directly the degrees
        else:
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                else:
                    charac_nu = left_basis.from_polynomial(charac_nu)
                
        # Do the tensor product with s[nu]
        if charac_nu != 0:
            charac += tensor([charac_nu, right_basis(s(nu))])
    return charac

Factorisation du produit tensoriel par représentation de $S_n$.

In [9]:
def factorise(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        print a
        show(b)

# Exemples de calculs

### Cas du partage 21

Calcul du déterminant de Vandermonde $\mathcal{V}_\lambda$ associé au partage $\lambda = 21$.

In [10]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Espace généré par $\mathcal{V}_\lambda$ et ses dérivées partielles.

In [11]:
P1 = DiagonalPolynomialRing(QQ, 3, 1, inert=1)
generator = {P1._grading_set((1,)) : [P1(v)]}
list_op = partial_derivatives(P1)
V1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V1.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection sur les composantes isotypiques.

In [12]:
V1_iso = Isotyp(V1, 3)
V1_iso.basis()

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,),
 ((0,), (2, 1)): (-theta00 + theta02,)}

On ajoute un jeu de variables supplémentaire et on polsarise.

In [13]:
P2 = DiagonalPolynomialRing(QQ, 3, 2, inert=1)
op_pol = polarization_operators(P2)
V1_pol = PolarizedSpace(P2, V1_iso, op_pol)
V1_pol.basis()

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((0, 0), (2, 1)): (theta00 - theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Calcul du bicaractère.

In [14]:
character(V1_pol, 3, 2)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Cas du diagramme 013 : pas ok 

In [46]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v

-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3

Créaction du sous-espace engendré par un détermiant de type Vandermonde, ses dérivées partielles.

In [62]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
generator = {P1._grading_set((v.degree(),)) : [P1(v)]}
#list_op = partial_derivatives(P1)
list_op = merge(merge(partial_derivatives(P1), steenrod_operators(P1, 1)), steenrod_operators(P1, 2))

In [63]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(2,): (-x00*x01 + x00*x02, -x00*x01 + x01*x02, x00^2 - x02^2, x01^2 - x02^2),
 (0,): (1,),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3),
 (1,): (x00, x01, x02),
 (4,): (x00^3*x01 - x00*x01^3 - x00^3*x02 + x01^3*x02 + x00*x02^3 - x01*x02^3,)}

In [65]:
character(Isotyp(W1, 3), 3, 1)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons p_1 et P_1 à v. 

In [91]:
P1.symmetric_derivative(P1(v), (1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [92]:
P1.steenrod_op(P1(v), 0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [67]:
P2 = DiagonalPolynomialRing(QQ, 3, 2)
op_pol = polarization_operators(P2)
W2 = PolarizedSpace(P2, Isotyp(W1, 3), op_pol)
character(W2, 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [101]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) 
                for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(P2, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(P2, list_degrees), 
#                       steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
character(Range(W2, sym_diff), 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1, 1] # s[1, 1, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [73]:
charac = character(W2, 3, 2) - character(Range(W2, sym_diff), 3, 2)
charac

s[1] # s[3] + s[1, 1] # s[2, 1] + s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [74]:
character(W2, 3, 2, left_basis=m) - character(Range(W2, sym_diff), 3, 2, left_basis=m)

m[1] # s[3] + 2*m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[1, 1, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [75]:
factorise(charac, 3)

[1, 1, 1]


s[2, 1] + s[4]

[3]


s[1]

[2, 1]


s[1, 1] + s[2] + s[3]

### Cas du diagramme 024 : ok en un jeu mais pas en 2

In [15]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
v

-x00^4*x01^2 + x00^2*x01^4 + x00^4*x02^2 - x01^4*x02^2 - x00^2*x02^4 + x01^2*x02^4

Créaction du sous-espace engendré par un détermiant de type Vandermonde, ses dérivées partielles et les opérateurs de Steenrod de degré 1 et 2.

In [16]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
generator = {P1._grading_set((v.degree(),)) : [P1(v)]}
list_steenrod = merge(steenrod_operators(P1,1), steenrod_operators(P1,2))
list_op = merge(partial_derivatives(P1), list_steenrod)

In [17]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00*x02, x01*x02, x00^2, x01^2, x02^2, x00*x01),
 (3,): (x01*x02^2 - 1/3*x02^3,
  x00*x02^2 - 1/3*x02^3,
  x00^2*x02 - 1/3*x02^3,
  x00*x01^2 - 1/3*x02^3,
  x00^2*x01 - 1/3*x02^3,
  x01^2*x02 - 1/3*x02^3,
  x01^3 - x02^3,
  x00^3 - x02^3),
 (4,): (-1/4*x00^4 + x00^3*x01 - x01^3*x02 + 1/4*x02^4,
  -1/4*x00^4 + x00*x01^3 - x01^3*x02 + 1/4*x02^4,
  -x01^3*x02 + x01*x02^3,
  -1/4*x00^4 + 1/4*x01^4 + x00^3*x02 - x01^3*x02,
  -1/4*x00^4 + 1/4*x01^4 - x01^3*x02 + x00*x02^3,
  1/6*x00^4 - x00^2*x01^2 + x01^2*x02^2 - 1/6*x02^4,
  -x00^2*x01^2 + 1/6*x01^4 + x00^2*x02^2 - 1/6*x02^4),
 (5,): (x00^2*x01^3 - 1/2*x00*x01^4 - 1/2*x00^4*x02 + 1/2*x01^4*x02 - x01^3*x02^2 + 1/2*x00*x02^4,
  -x00^3*x01^2 + 1/2*x00*x01^4 + x00^3*x02^2 - 1/2*x00*x02^4,
  1/2*x00^4*x02 - 1/2*x01^4*x02 - x00^2*x02^3 + x01^2*x02^3,
  x00^4*x01 - x00*x01^4 - x00^4*x02 + x01^4*x02 + x00*x02^4 - x01*x02^4),
 (6,): (x00^4*x01^2 - x00^2*x01^4 - x00^4*x02^2 + x01^4*x02^2 + x00^2*x02^4 - x01^2*x02^4,)}

On demande uniquement la composante isotypique de type $111$.

In [18]:
Isotyp(V1, Partition([1,1,1])).basis()

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

In [19]:
P2 = DiagonalPolynomialRing(QQ, 3, 2)
list_steenrod = merge(steenrod_operators(P2,1), steenrod_operators(P2,2))
op_pol = merge(polarization_operators(P2), list_steenrod)
W2 = PolarizedSpace(P2, Isotyp(W1, 3), op_pol)
character(W2, 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + s[3] # s[3] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 3*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

Calcul de l'idéal pour quotienter. 

In [20]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
sym_diff = symmetric_derivatives(P2, list_degrees)
range_op = merge(merge(sym_diff, steenrod_operators(P2, degree=1)), steenrod_operators(P2, degree=2))
character(Range(W2, range_op), 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 2*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1]

In [21]:
charac = character(W2, 3, 2) - character(Range(W2, range_op), 3, 2)
charac

s[3] # s[3] + s[4] # s[2, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

Ou dans la base des monomiales

In [22]:
character(W2, 3, 2, left_basis=m) - character(Range(W2, range_op), 3, 2, left_basis=m)

m[2, 1] # s[3] + m[2, 2] # s[2, 1] + m[3] # s[3] + m[3, 1] # s[2, 1] + m[3, 2] # s[2, 1] + m[3, 3] # s[1, 1, 1] + m[4] # s[2, 1] + m[4, 1] # s[2, 1] + m[4, 2] # s[1, 1, 1] + m[5] # s[2, 1] + m[5, 1] # s[1, 1, 1] + m[6] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [23]:
factorise(charac, 3)

[1, 1, 1]


s[6]

[3]


s[3]

[2, 1]


s[4] + s[5]

### Cas du partage 311

In [24]:
P1 = DiagonalPolynomialRing(QQ, 5, 1, inert=1)
v = vandermonde(Partition([3,1,1]))
deg_v = 3
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

P2 = DiagonalPolynomialRing(QQ, 5, 2, inert=1)
op_pol = polarization_operators(P2)
W2 = PolarizedSpace(P2, Isotyp(W1, 5), op_pol)
character(W2, 5, 2)

s[] # s[3, 1, 1] + s[1] # s[2, 1, 1, 1] + s[1] # s[2, 2, 1] + s[1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 1, 1, 1] + s[3] # s[1, 1, 1, 1, 1]

### Cas du diagramme $\{(0,0),(0,1),(1,1),(0,2)\}$

Pas de quotient, ni de polarisation dans ce cas.

In [25]:
P1 = DiagonalPolynomialRing(QQ, 4, 1, inert=1)
v = vandermonde(Diagram([(0,0),(0,1),(1,1),(0,2)]))
show(v)
deg_v = 1
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
character(Isotyp(W1, 4), 4, 1)

x01*theta00^2*theta01*theta02 - x02*theta00^2*theta01*theta02 - x00*theta00*theta01^2*theta02 + x02*theta00*theta01^2*theta02 + x00*theta00*theta01*theta02^2 - x01*theta00*theta01*theta02^2 - x01*theta00^2*theta01*theta03 + x03*theta00^2*theta01*theta03 + x00*theta00*theta01^2*theta03 - x03*theta00*theta01^2*theta03 + x02*theta00^2*theta02*theta03 - x03*theta00^2*theta02*theta03 - x02*theta01^2*theta02*theta03 + x03*theta01^2*theta02*theta03 - x00*theta00*theta02^2*theta03 + x03*theta00*theta02^2*theta03 + x01*theta01*theta02^2*theta03 - x03*theta01*theta02^2*theta03 - x00*theta00*theta01*theta03^2 + x01*theta00*theta01*theta03^2 + x00*theta00*theta02*theta03^2 - x02*theta00*theta02*theta03^2 - x01*theta01*theta02*theta03^2 + x02*theta01*theta02*theta03^2

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]